<div align="center">

###### Lab 2

# National Tsing Hua University

#### Spring 2025

#### 11320IEEM 513600

#### Deep Learning and Industrial Applications
    
## Lab 2: Predicting Heart Disease with Deep Learning

</div>

### Introduction

In the realm of healthcare, early detection and accurate prediction of diseases play a crucial role in patient care and management. Heart disease remains one of the leading causes of mortality worldwide, making the development of effective diagnostic tools essential. This lab leverages deep learning to predict the presence of heart disease in patients using a subset of 14 key attributes from the Cleveland Heart Disease Database. The objective is to explore and apply deep learning techniques to distinguish between the presence and absence of heart disease based on clinical parameters.

Throughout this lab, you'll engage with the following key activities:
- Use [Pandas](https://pandas.pydata.org) to process the CSV files.
- Use [PyTorch](https://pytorch.org) to build an Artificial Neural Network (ANN) to fit the dataset.
- Evaluate the performance of the trained model to understand its accuracy.

### Attribute Information

1. age: Age of the patient in years
2. sex: (Male/Female)
3. cp: Chest pain type (4 types: low, medium, high, and severe)
4. trestbps: Resting blood pressure
5. chol: Serum cholesterol in mg/dl
6. fbs: Fasting blood sugar > 120 mg/dl
7. restecg: Resting electrocardiographic results (values 0,1,2)
8. thalach: Maximum heart rate achieved
9. exang: Exercise induced angina
10. oldpeak: Oldpeak = ST depression induced by exercise relative to rest
11. slope: The slope of the peak exercise ST segment
12. ca: Number of major vessels (0-3) colored by fluoroscopy
13. thal: 3 = normal; 6 = fixed defect; 7 = reversible defect
14. target: target have disease or not (1=yes, 0=no)

### References
- [UCI Heart Disease Data](https://www.kaggle.com/datasets/redwankarimsony/heart-disease-data) for the dataset we use in this lab.


## A. Checking and Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/深度學習/heart_dataset_train_all.csv')
df

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
# checking for null values
df.isnull().sum()

In [ ]:
df = df.dropna()

In [ ]:
df.shape

In [ ]:
# Mapping 'sex' descriptions to numbers
sex_description = {
    'Male': 0,
    'Female': 1,
}
df.loc[:, 'sex'] = df['sex'].map(sex_description)

# Mapping 'cp' (chest pain) descriptions to numbers
pain_description = {
    'low': 0,
    'medium': 1,
    'high': 2,
    'severe': 3
}
df.loc[:, 'cp'] = df['cp'].map(pain_description)

df

In [ ]:
df.describe()

In [ ]:
df.corr()

#### Converting the DataFrame to a NumPy Array

In [ ]:
import numpy as np

np_data = df.values
np_data.shape

In [ ]:
split_point = int(np_data.shape[0]*0.7)

np.random.shuffle(np_data)

x_train = np_data[:split_point, :13]
y_train = np_data[:split_point, 13]
x_val = np_data[split_point:, :13]
y_val = np_data[split_point:, 13]

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert to PyTorch tensors
x_train = np.array(x_train, dtype=float)
x_train = torch.from_numpy(x_train).float()
y_train = np.array(y_train, dtype=int)
y_train = torch.from_numpy(y_train).long()

x_val = np.array(x_val, dtype=float)
x_val = torch.from_numpy(x_val).float()
y_val = np.array(y_val, dtype=int)
y_val = torch.from_numpy(y_val).long()

batch_size = 32

# Create datasets
train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f'Number of samples in train and validation are {len(train_loader.dataset)} and {len(val_loader.dataset)}.')

## B. Defining Neural Networks

In PyTorch, we can use **class** to define our custom neural network architectures by subclassing the `nn.Module` class. This gives our neural network all the functionality it needs to work with PyTorch's other utilities and keeps our implementation organized.

- Neural networks are defined by subclassing `nn.Module`.
- The layers of the neural network are initialized in the `__init__` method.
- The forward pass operations on input data are defined in the `forward` method.

It's worth noting that while we only define the forward pass, PyTorch will automatically derive the backward pass for us, which is used during training to update the model's weights."

In [ ]:
import torch.nn as nn

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(13, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 2)
        ).cuda()

    def forward(self, x):
        return self.model(x)

## C. Training the Neural Network

In [ ]:
# Check your GPU status.
!nvidia-smi

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR
from tqdm.auto import tqdm

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

epochs = 100

model = Model()
# print(model)

best_val_loss = float('inf')
best_val_acc = -1

criterion = nn.CrossEntropyLoss()
# change learning rate
optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = optim.Adam(model.parameters(), lr=1e-3)
#optimizer = optim.Adam(model.parameters(), lr=1e-3)

lr_scheduler = CosineAnnealingLR(optimizer, T_max=epochs, eta_min=0)

for epoch in tqdm(range(epochs)):
    # Training
    model.train()
    total_loss = 0.0
    train_correct = 0
    total_train_samples = 0

    for features, labels in train_loader:
        features = features.cuda()
        labels = labels.cuda()

        outputs = model(features)

        loss = criterion(outputs, labels)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_predicted = outputs.argmax(-1)
        train_correct += (train_predicted == labels).sum().item()
        total_train_samples += labels.size(0)

    # Learning rate update
    lr_scheduler.step()

    avg_train_loss = total_loss / len(train_loader)
    train_accuracy = 100. * train_correct / total_train_samples

    # Validation
    model.eval()
    total_val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in val_loader:
            features = features.cuda()
            labels = labels.cuda()

            outputs = model(features)

            loss = criterion(outputs, labels)
            total_val_loss += loss.item()

            predicted = outputs.argmax(-1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = 100. * correct / total

    # Checkpoint
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss

    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        torch.save(model.state_dict(), 'model_classification.pth')

    print(f'Epoch {epoch+1}/{epochs}, Train loss: {avg_train_loss:.4f}, Train acc: {train_accuracy:.4f}%, Val loss: {avg_val_loss:.4f}, Val acc: {val_accuracy:.4f}%, Best Val loss: {best_val_loss:.4f} Best Val acc: {best_val_acc:.2f}%')

    # Store performance
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)

In [ ]:
# 超參數-learning rate 0.1 0.01 0.001
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm.auto import tqdm
import pandas as pd

epochs = 100
learning_rates = [0.001, 0.01, 0.1]  # 測試不同學習率
results = []

for lr in learning_rates:
    print(f'\nTraining with learning rate: {lr}')

    # 初始化模型 & 優化器
    model = Model().cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    lr_scheduler = CosineAnnealingLR(optimizer, T_max=epochs, eta_min=0)

    best_val_loss = float('inf')
    best_val_acc = -1

    for epoch in tqdm(range(epochs)):
        # Training
        model.train()
        total_loss = 0.0
        train_correct = 0
        total_train_samples = 0

        for features, labels in train_loader:
            features, labels = features.cuda(), labels.cuda()

            outputs = model(features)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_predicted = outputs.argmax(-1)
            train_correct += (train_predicted == labels).sum().item()
            total_train_samples += labels.size(0)

        # Learning rate update
        lr_scheduler.step()

        avg_train_loss = total_loss / len(train_loader)
        train_accuracy = 100. * train_correct / total_train_samples

        # Validation
        model.eval()
        total_val_loss = 0.0
        val_correct = 0
        total_val_samples = 0

        with torch.no_grad():
            for features, labels in val_loader:
                features, labels = features.cuda(), labels.cuda()
                outputs = model(features)
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()
                predicted = outputs.argmax(-1)
                val_correct += (predicted == labels).sum().item()
                total_val_samples += labels.size(0)

        avg_val_loss = total_val_loss / len(val_loader)
        val_accuracy = 100. * val_correct / total_val_samples

        # Checkpoint
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss

        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            torch.save(model.state_dict(), f'model_lr_{lr}.pth')

        print(f'Epoch {epoch+1}/{epochs}, Train loss: {avg_train_loss:.4f}, Train acc: {train_accuracy:.2f}%, '
              f'Val loss: {avg_val_loss:.4f}, Val acc: {val_accuracy:.2f}%')

    # 測試集評估
    model.eval()
    total_test_loss = 0.0
    test_correct = 0
    total_test_samples = 0

    with torch.no_grad():
        for features, labels in test_loader:
            features, labels = features.cuda(), labels.cuda()
            outputs = model(features)
            loss = criterion(outputs, labels)
            total_test_loss += loss.item()
            predicted = outputs.argmax(-1)
            test_correct += (predicted == labels).sum().item()
            total_test_samples += labels.size(0)

    avg_test_loss = total_test_loss / len(test_loader)
    test_accuracy = 100. * test_correct / total_test_samples

    # 存入結果
    results.append([lr, avg_train_loss, avg_val_loss, avg_test_loss, train_accuracy, val_accuracy, test_accuracy])

# 轉成 DataFrame，輸出表格
columns = ["Learning Rate", "Train Loss", "Validation Loss", "Test Loss", "Train Accuracy", "Validation Accuracy", "Test Accuracy"]
df = pd.DataFrame(results, columns=columns)
print(df)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Draw plots for Train and Validation Losses
plt.figure(figsize=(12, 6))

# Plot Train Loss
plt.subplot(1, 2, 1)
sns.lineplot(data=df, x='Learning Rate', y='Train Loss', marker='o', hue='Learning Rate')
plt.title("Train Loss vs Learning Rate")
plt.xlabel("Learning Rate")
plt.ylabel("Train Loss")

# Plot Validation Loss
plt.subplot(1, 2, 2)
sns.lineplot(data=df, x='Learning Rate', y='Validation Loss', marker='o', hue='Learning Rate')
plt.title("Validation Loss vs Learning Rate")
plt.xlabel("Learning Rate")
plt.ylabel("Validation Loss")

plt.tight_layout()
plt.show()

# Draw plots for Train and Validation Accuracy
plt.figure(figsize=(12, 6))

# Plot Train Accuracy
plt.subplot(1, 2, 1)
sns.lineplot(data=df, x='Learning Rate', y='Train Accuracy', marker='o', hue='Learning Rate')
plt.title("Train Accuracy vs Learning Rate")
plt.xlabel("Learning Rate")
plt.ylabel("Train Accuracy")

# Plot Validation Accuracy
plt.subplot(1, 2, 2)
sns.lineplot(data=df, x='Learning Rate', y='Validation Accuracy', marker='o', hue='Learning Rate')
plt.title("Validation Accuracy vs Learning Rate")
plt.xlabel("Learning Rate")
plt.ylabel("Validation Accuracy")

plt.tight_layout()
plt.show()


In [ ]:
#超參數-epoch 50 100 150
epochs_list = [50, 100, 150]  # 測試不同的 epoch 值
learning_rates = [0.001]  # 仍然測試不同學習率
results = []

for epochs in epochs_list:
    for lr in learning_rates:
        print(f'\\nTraining with Learning Rate: {lr}, Epochs: {epochs}')

        # 初始化模型 & 優化器
        model = Model().cuda()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)
        lr_scheduler = CosineAnnealingLR(optimizer, T_max=epochs, eta_min=0)

        best_val_loss = float('inf')
        best_val_acc = -1

        for epoch in tqdm(range(epochs)):
            # Training
            model.train()
            total_loss = 0.0
            train_correct = 0
            total_train_samples = 0

            for features, labels in train_loader:
                features, labels = features.cuda(), labels.cuda()
                outputs = model(features)
                loss = criterion(outputs, labels)
                total_loss += loss.item()

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                train_predicted = outputs.argmax(-1)
                train_correct += (train_predicted == labels).sum().item()
                total_train_samples += labels.size(0)

            # Learning rate update
            lr_scheduler.step()

            avg_train_loss = total_loss / len(train_loader)
            train_accuracy = 100. * train_correct / total_train_samples

            # Validation
            model.eval()
            total_val_loss = 0.0
            val_correct = 0
            total_val_samples = 0

            with torch.no_grad():
                for features, labels in val_loader:
                    features, labels = features.cuda(), labels.cuda()
                    outputs = model(features)
                    loss = criterion(outputs, labels)
                    total_val_loss += loss.item()
                    predicted = outputs.argmax(-1)
                    val_correct += (predicted == labels).sum().item()
                    total_val_samples += labels.size(0)

            avg_val_loss = total_val_loss / len(val_loader)
            val_accuracy = 100. * val_correct / total_val_samples

            # Checkpoint
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss

            if val_accuracy > best_val_acc:
                best_val_acc = val_accuracy
                torch.save(model.state_dict(), f'model_lr_{lr}_epochs_{epochs}.pth')

            print(f'Epoch {epoch+1}/{epochs}, Train loss: {avg_train_loss:.4f}, Train acc: {train_accuracy:.2f}%, '
                  f'Val loss: {avg_val_loss:.4f}, Val acc: {val_accuracy:.2f}%')

        # 測試集評估
        model.eval()
        total_test_loss = 0.0
        test_correct = 0
        total_test_samples = 0

        with torch.no_grad():
            for features, labels in test_loader:
                features, labels = features.cuda(), labels.cuda()
                outputs = model(features)
                loss = criterion(outputs, labels)
                total_test_loss += loss.item()
                predicted = outputs.argmax(-1)
                test_correct += (predicted == labels).sum().item()
                total_test_samples += labels.size(0)

        avg_test_loss = total_test_loss / len(test_loader)
        test_accuracy = 100. * test_correct / total_test_samples

        # 存入結果
        results.append([lr, epochs, avg_train_loss, avg_val_loss, avg_test_loss, train_accuracy, val_accuracy, test_accuracy])

# 轉成 DataFrame，輸出表格
columns = ["Learning Rate", "Epochs", "Train Loss", "Validation Loss", "Test Loss", "Train Accuracy", "Validation Accuracy", "Test Accuracy"]
df = pd.DataFrame(results, columns=columns)

# 調整 Accuracy 格式為 xx.xx%
df["Train Accuracy"] = df["Train Accuracy"].map(lambda x: f"{x:.2f}%")
df["Validation Accuracy"] = df["Validation Accuracy"].map(lambda x: f"{x:.2f}%")
df["Test Accuracy"] = df["Test Accuracy"].map(lambda x: f"{x:.2f}%")

print(df)


In [ ]:
# 畫出損失和準確度曲線
plt.figure(figsize=(12, 6))

# 損失曲線
plt.subplot(1, 2, 1)
plt.plot(range(epochs * len(epochs_list)), train_losses, label='Train Loss')
plt.plot(range(epochs * len(epochs_list)), val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train vs Validation Loss')
plt.legend()

# 準確度曲線
plt.subplot(1, 2, 2)
plt.plot(range(epochs * len(epochs_list)), train_accuracies, label='Train Accuracy')
plt.plot(range(epochs * len(epochs_list)), val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Train vs Validation Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

#### Visualizing the model performance

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# Plotting training and validation accuracy
ax[0].plot(train_accuracies)
ax[0].plot(val_accuracies)
ax[0].set_title('Model Accuracy')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].legend(['Train', 'Val'])

# Plotting training and validation loss
ax[1].plot(train_losses)
ax[1].plot(val_losses)
ax[1].set_title('Model Loss')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')
ax[1].legend(['Train', 'Val'])

plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# Plotting training and validation accuracy
ax[0].plot(train_accuracies)
ax[0].plot(val_accuracies)
ax[0].set_title('Model Accuracy')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].legend(['Train', 'Val'])

# Plotting training and validation loss
ax[1].plot(train_losses)
ax[1].plot(val_losses)
ax[1].set_title('Model Loss')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')
ax[1].legend(['Train', 'Val'])

plt.show()

## D. Evaluating Your Trained Model

In [ ]:
# read test file
test_data = pd.read_csv('/content/drive/MyDrive/深度學習/heart_dataset_test.csv')
test_data.head()

In [ ]:
test_data.isnull().sum()

In [ ]:
test_data = test_data.values
test_data.shape

In [ ]:
# Convert to PyTorch tensors
x_test = torch.from_numpy(test_data[:, :13]).float()
y_test = torch.from_numpy(test_data[:, 13]).long()

# Create datasets
test_dataset = TensorDataset(x_test, y_test)

# Create dataloaders
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
# Load the trained weights
model.load_state_dict(torch.load('model_classification.pth'))

# Set the model to evaluation mode
model.eval()

test_correct = 0
test_total = 0

with torch.no_grad():
    for features, labels in test_loader:

        features = features.cuda()
        labels = labels.cuda()

        outputs = model(features)

        predicted = outputs.argmax(-1)
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

print(f'Test accuracy is {100. * test_correct / test_total}%')